Вычислить с заданой погрешностью решение задачи Коши

$\left\{ \begin{gathered} \frac{dy}{dx} = -\frac{y^2 + 4x(x + 1)}{y} \\ y(1) = 12 \end{gathered}\right. ,\space \epsilon = 10^{-4}, \space x \in (1, 2)$

Аналитическое решение данного уравнения:

$y(x) = 2 \cdot \sqrt{x^2 + 35e^{2 - 2x}} $



In [17]:
import numpy as np
import math

In [18]:
def solve(x):
    return 2*np.sqrt(x**2 + 35*np.e**(2 - 2*x))

solve(2)

5.911593664412819

In [19]:
a = 1
b = 2
eps = 1e-4
k = 1
y0 = 12
target = np.linspace(a, b, 11)

def f(x,y):
    if y != 0:
        return -(y**2 + 4*x*(x + 1))/y
    else:
        print('error')
        return 0

In [20]:
def genX(n):
    xx = []
    for i in range(10):
        xx.extend(list(np.linspace(target[i], target[i+1], n, endpoint=False)))

    xx.append(b)
    
    return xx
    
def getStep(xx):
    h = (b-a)/(len(xx)-1)
    return h

In [21]:
def iterate(x, y, h):
    f1 = f(x,y)
    f2 = f(x + h/3, y + h/3*f1)
    f3 = f(x + 2*h/3, y + 2*h/3*f2)
    return y + h/4*(f1 + 3*f3)

In [22]:
def rk(xx, h):
    x = a
    y_prev = y0
    y_curr = y0

    res = []
    for x in xx:
        for t in target:
            if abs(t-x)<1e-8:
                res.append(y_curr)
        y_prev = y_curr
        y_curr = iterate(x, y_curr, h)

        yh = iterate(x, y_curr, h)
        y2h = iterate(x, y_curr, 2*h)
        # assert abs(y2h - yh) <= (2**k - 1) * eps
    return res

In [ ]:
xx = genX(4096)
res = rk(xx, getStep(xx))

In [ ]:
xxmax = genX(2**20)
resmax = rk(xxmax, getStep(xxmax))

In [ ]:
len(xxmax)

10485761

In [ ]:
diff = []
for i in range(len(res)):
    diff.append((res[i]-resmax[i]))

In [ ]:
import pandas as pd
pd.DataFrame({'x': target, 'y': res, 'y*': resmax, 'd': diff}).transpose()

,0,1,2,3,4,5,6,7,8,9,10
x,1.0,1.100000e+00,1.200000e+00,1.300000e+00,1.400000e+00,1.500000e+00,1.600000e+00,1.700000e+00,1.800000e+00,1.900000e+00,2.000000e+00
y,12.0,1.092988e+01,9.980221e+00,9.142955e+00,8.411067e+00,7.778375e+00,7.239281e+00,6.788488e+00,6.420710e+00,6.130403e+00,5.911594e+00
y*,12.0,1.092988e+01,9.980221e+00,9.142955e+00,8.411067e+00,7.778375e+00,7.239281e+00,6.788488e+00,6.420710e+00,6.130403e+00,5.911594e+00
d,0.0,4.156675e-13,7.158718e-13,7.105427e-13,-4.831691e-13,-1.456613e-13,-1.900702e-13,-5.053735e-13,-8.189005e-13,-9.166001e-13,-9.245937e-13


In [ ]:
N = []
d = []

for i in range(18):
    xx = genX(2**i)
    res = rk(xx, getStep(xx))
    norm = 0
    for j in range(len(res)):
        norm = max(abs(res[j]-resmax[j]), norm)
    d.append(norm)
    N.append(2**i)

In [ ]:
rate = [0]
for i in range(len(d)-1):
    rate.append(d[i]/d[i+1])

rate

[0,
 8.379702660289992,
 8.189338504107182,
 8.094478243091993,
 8.047097662658269,
 8.022813079879171,
 8.005638011355972,
 7.956756541848614,
 7.630346850035393,
 5.823910482921083,
 2.5174203113417346,
 1.2319634703196347,
 1.0518731988472623,
 0.9621072088724584,
 1.0207547169811322,
 0.9540954095409541,
 1.0354147250698975,
 0.8298530549110595]

In [ ]:
pd.DataFrame({'N': N, 'd': d, 'rate': rate})

,N,d,rate
0,1,3.061901e-04,0.000000
1,2,3.653949e-05,8.379703
2,4,4.461837e-06,8.189339
3,8,5.512198e-07,8.094478
4,16,6.849921e-08,8.047098
5,32,8.538054e-09,8.022813
6,64,1.066505e-09,8.005638
7,128,1.340377e-10,7.956757
8,256,1.756639e-11,7.630347
9,512,3.016254e-12,5.823910


In [ ]:
norm = max(diff)
norm

7.158718062783009e-13